### 랭체인으로 멀티턴 대화하기(이전 대화 내용 기억하도록)

In [2]:
# 랭체인에서 제공하는 메시지 히스토리 활용하기
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 메모리 내에서 메시지를 리스트 형태로 보관합니다. 앱 종료시 삭제
from langchain_core.chat_history import InMemoryChatMessageHistory
# 모델 생성 시 대화 기록을 함께 전달 가능
from langchain_core.runnables.history import RunnableWithMessageHistory

model = ChatOpenAI(model="gpt-4o-mini")

# session_id를 기준으로 사용자의 대화 기록을 저장하는 딕셔너리
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [ ]:
config = {"configurable": {"session_id": "abc2"}} # config로 제공한 session_id에 따라 고유의 대화 내용이 기록

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 이성용이야")],
    config=config,
)

print(response.content)

안녕하세요, 이성용님! 만나서 반갑습니다. 어떻게 도와드릴까요?


In [4]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 이성용입니다! 제가 도와드릴 일이 있을까요?


In [5]:
# 세션id를 바꿀 경우 이름을 기억하지 못함
config = {"configurable": {"session_id": "abc3"}} 

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

저는 당신의 이름을 알 수 없습니다. 하지만 당신이 원하신다면 이름을 알려주시면 반갑게 대화할 수 있습니다!


In [6]:
# 세션id를 원래대로 돌리면 기억함
config = {"configurable": {"session_id": "abc2"}} 

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 이성용입니다. 더 궁금한 점이 있으신가요?


In [8]:
config = {"configurable": {"session_id": "abc2"}} 

# invoke를 stream으로 바꾸면 스트림 방식으로 출력
for r in with_message_history.stream(
    [HumanMessage(content="내가 어느 사람인지 맞춰보고, 그 나라의 국가를 불러줘.")],
    config=config,
):
    print(r.content, end="|")


|이|성|용|님|에| 대한| 구|체|적인| 정보|는| 알| 수| 없|지만|,| 이름|에서| 유|추|해|보|면| 한국|분|이|실| 가능|성이| 높|습니다|.| 그렇|다면| 한국|의| 국가|인| "|애|국|가|"|를| 언|급|할| 수| 있|겠습니다|.| 

|"|애|국|가|"|의| 가|사| 내용|은| 조|국|에| 대한| 사랑|과| 자|긍|심|을| 담|고| 있습니다|.| 더| 궁|금|한| 점|이| 있으|시면| 말씀|해| 주세요|!||